In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.1/spark-3.0.1-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-3.0.1-bin-hadoop2.7.tgz
!pip install -q findspark

In [ ]:
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content//spark-3.0.1-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()


In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import * 

In [ ]:
import numpy as np
import pandas as pd
from io import StringIO

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pyspark
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('SparkProject').getOrCreate()

In [ ]:
spark

## 2. Define the schema for all the dataset and use the schema to read the file as DF.

In [ ]:
from pyspark.sql.types import * 

Schema_OD = StructType([
      StructField('Order_channel_code',IntegerType(),True),
      StructField('Order_channel_type',StringType(),True),    
])


Schema_PO = StructType([
      StructField('Retailer_code',IntegerType(),True),
      StructField('product_number',IntegerType(),True),
      StructField('Date',StringType(),True),
      StructField('Quantity',IntegerType(),True)
])


Schema_P = StructType([
      StructField('product_number',IntegerType(),True),
      StructField('Product_line',StringType(),True),
      StructField('Product_type',StringType(),True),
      StructField('Product',StringType(),True),
      StructField('Product_brand',StringType(),True),
      StructField('Product_colour',StringType(),True),
      StructField('Unit_loss',FloatType(),True),
      StructField('Unit_price',FloatType(),True),
])


Schema_S = StructType([
      StructField('Retailer_code',IntegerType(),True),
      StructField('Product_Number',IntegerType(),True),
      StructField('Order_channel_code',IntegerType(),True),
      StructField('Date',StringType(),True),
      StructField('Quantity',IntegerType(),True),
      StructField('Unit_price',FloatType(),True),
      StructField('Unit_sale_price',FloatType(),True),
])

Schema_RD = StructType([
      StructField('Retailer_code',IntegerType(),True),
      StructField('Retailer_name',StringType(),True),
      StructField('Type',StringType(),True),
      StructField('Country',StringType(),True),
])

3. Using Spark SQL read the raw data from the files systems (HDFS), as a separate data
frames by specifying the schema for reading the files. (10 Marks)

In [ ]:
OrderChannelsDF = spark.read.format('csv').option('header','true').load('/content/drive/MyDrive/Project/Dataset/Dataset/order_channels.csv')

In [ ]:
OrderChannelsDF.show()

In [ ]:
OrderChannelsDF.printSchema()

In [ ]:
ProductsOrdersDF = spark.read.format('csv').option('header','true').load('/content/drive/My Drive/Project/Dataset/Dataset/products_orders.csv')

In [ ]:
ProductsOrdersDF.show(5)

In [ ]:
ProductsOrdersDF.printSchema()

In [ ]:
ProductsDF = spark.read.format('csv').option('header','true').load('/content/drive/My Drive/Project/Dataset/Dataset/products.csv')

In [ ]:
ProductsDF.show(5)

In [ ]:
ProductsDF.printSchema()

In [ ]:
RetailersDetailsDF = spark.read.format('csv').option('header','true').load('/content/drive/My Drive/Project/Dataset/Dataset/retailers_details.csv')

In [ ]:
RetailersDetailsDF.show(5)

In [ ]:
RetailersDetailsDF.printSchema()

In [ ]:
salesDF = spark.read.format('csv').option('header','true').load('/content/drive/My Drive/Project/Dataset/Dataset/sales.csv')

In [ ]:
salesDF.show(5)

In [ ]:
salesDF.printSchema()

# 4. Perform a basic analysis of the various data frames and store the results on to the file system
#Basic Analysis include: Select any one DF and show the following
# a. Display the columns names (3 Marks)

In [ ]:
salesDF.columns

## #b. Display the datatypes of the columns (3 Marks)

In [ ]:
salesDF.dtypes

#c. Find the maximum and minimum values in each column (3 Marks)

In [ ]:
salesDF.select(max('Unit_price'),max('Unit_sale_price')).show()

In [ ]:
salesDF.select(min('Unit_price'),min('Unit_sale_price')).show()

## #d. Define a table/view on the spark dataframe created to run sql queries on the dataframe. (5 Marks)

In [ ]:
salesDF.createOrReplaceTempView('sales_temp')
sales_temp1=spark.sql("SELECT * FROM sales_temp")
sales_temp1.show()

#d. Define a table/view on the spark dataframe created to run sql queries on the dataframe. (5 Marks)

In [ ]:
sales=salesDF.toPandas()

In [ ]:
sales


In [ ]:
sales.isnull().sum()

# Analyze the datasets by merging various data frames. Try to find the answers
## for the questions for example:
# f. How many orders chose a particular channel? (10 marks)

In [ ]:
salesDF.groupBy('Order_channel_code').agg({'Quantity':'sum'}).show()

## g. Show country wise retailer names. (10 marks)

In [ ]:
Retailer_sorted1=RetailersDetailsDF.select('Country','Retailer_name').show()

## h. Number of products available under each project type.

In [ ]:
ProductsDF.groupBy('Product_type').agg({'Product':'count'}).show()

## i. Find minimum and maximum unit_price for product_type. (5 marks)

In [ ]:
from pyspark.sql import functions as F
ProductsDF.groupBy('Product_type').agg(F.max(ProductsDF.Unit_price),F.min(ProductsDF.Unit_price)).show()